In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ln -s /content/drive/My\ Drive/People_Detection/dadosImagem /content/Aula

In [0]:
!ls Aula/Teste

negativos  positivos


In [0]:
!pip uninstall opencv-python -y

Uninstalling opencv-python-3.4.7.28:
  Successfully uninstalled opencv-python-3.4.7.28


In [0]:
!pip install opencv-contrib-python==3.4.2.17 --force-reinstall

     |████████████████████████████████| 30.6MB 20.2MB/s 
     |████████████████████████████████| 20.0MB 177kB/s 
ERROR: tensor2tensor 1.14.1 requires opencv-python, which is not installed.
ERROR: stable-baselines 2.2.1 requires opencv-python, which is not installed.
ERROR: imgaug 0.2.9 requires opencv-python, which is not installed.
ERROR: dopamine-rl 1.0.5 requires opencv-python>=3.4.1.15, which is not installed.
ERROR: albumentations 0.1.12 requires opencv-python, which is not installed.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.17.3
    Uninstalling numpy-1.17.3:
      Successfully uninstalled numpy-1.17.3
  Found existing installation: opencv-contrib-python 3.4.3.18
    Uninstalling opencv-contrib-python-3.4.3.18:
      Successfully uninstalled opencv-contrib-p

In [0]:
import cv2

class Descritor:

    @staticmethod
    def get_SIFT_descritores(img_caminho):
        
        LARGURA = 360
        ALTURA = 360
        
        # Ler a imgem
        img_teste = cv2.imread(img_caminho, 0)
        
        #Redimensionar
        img_redimensionada = cv2.resize(img_teste, (LARGURA, ALTURA), interpolation=cv2.INTER_CUBIC)
        
        # Remover ruídos
        img_equalizada = cv2.equalizeHist(img_redimensionada)
        img_suavizada = cv2.GaussianBlur(img_equalizada, (9,9), 1)
        
        #Detectar pontos chaves
        sift = cv2.xfeatures2d.SIFT_create(nfeatures=512)
        pontos_chave = sift.detect(img_suavizada, None)
        
        pontos_chave, descritores = sift.compute(img_suavizada, pontos_chave)
        
        return descritores

    @staticmethod
    def salvar_descritor(descritor, caminho, nome_arquivo):
        descritor = descritor.reshape((1,descritor.size))
        arquivo = open(os.path.join(caminho, nome_arquivo), 'a')
        np.savetxt(arquivo, descritor, delimiter=',', fmt='%i')
        arquivo.close()
    
    @staticmethod
    def carregar_descritores(caminho, nome_arquivo='orb_descritor.csv'):
        descritores = np.loadtxt(os.path.join(caminho, nome_arquivo), delimiter=',')
        print('formato do array de descritores: ', descritores.shape)
        return descritores

In [0]:
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
import os
import numpy as np

In [0]:
QUANTIDADE_PALAVRAS_VIRTUAIS = 128

class PacoteDePalavras:
    
    def gerar_dicionario(self, lista_descritores):
        kmeans = KMeans(n_clusters = QUANTIDADE_PALAVRAS_VIRTUAIS)
        kmenas = kmeans.fit(lista_descritores)
        self.dicionario = kmeans.cluster_centers_
        
    def histograma_de_frequencia(self, descritor):

        try:
            algoritmo_knn = NearestNeighbors(n_neighbors = 1)
            algoritmo_knn.fit(self.dicionario)
            mais_proximos = algoritmo_knn.kneighbors(descritor, return_distance = False).flatten()

            histograma_caracteristica = np.histogram(mais_proximos, bins=np.arange(self.dicionario.shape[0]+1))[0]
        
            return histograma_caracteristica
        except AttributeError:
            print("O atributo dicionario nao foi definido")
            
    def salvar_dicionario(self, caminho='', nome_dicionario = 'dicionario.csv'):
        try:
            np.savetxt(os.path.join(caminho, nome_dicionario), self.dicionario, delimiter=',', fmt='%f')
            print("Dicionario salvo")
            
        except AttributeError:
            print("Dicionario Vazio")

    def carregar_dicionario(self, caminho='', nome_dicionario = 'dicionario.csv'):

        self.dicionario = np.loadtxt(os.path.join(caminho,nome_dicionario), delimiter=',')
    

In [0]:
def arquivo_diretorio(caminho):

    for raiz,diretorios,arquivos in os.walk(caminho):
        for arquivo in arquivos:
            if arquivo.endswith('.png'):
                yield arquivo

In [0]:
DICIONARIO_NOME = 'dicionario.csv'
dados_treinamento = ['Aula/Treinamento/positivos/', 'Aula/Treinamento/negativos']

# Rotina para criação do dicionario de palavras virtuais

descritores = np.empty((0,128), dtype=np.uint8)

for caminho in dados_treinamento:

    for arquivo in arquivo_diretorio(caminho):
        sift_descritor = Descritor.get_SIFT_descritores(os.path.join(caminho,arquivo))
        descritores = np.append(descritores, sift_descritor, axis=0)
                
img_representacao = PacoteDePalavras()
img_representacao.gerar_dicionario(descritores)
img_representacao.salvar_dicionario('Aula/', DICIONARIO_NOME)

In [0]:
DICIONARIO_NOME = 'dicionario.csv'
img_representacao = PacoteDePalavras()
img_representacao.carregar_dicionario('Aula/', DICIONARIO_NOME)

In [0]:
NOME_DESCRITOR = 'sift_descritor.csv'

for caminho in dados_treinamento:

    caminho_descritor = os.path.join(caminho, NOME_DESCRITOR)
    if os.path.exists(caminho_descritor):
        os.remove(caminho_descritor)

    for arquivo in arquivo_diretorio(caminho):
        descritor = Descritor.get_SIFT_descritores(os.path.join(caminho, arquivo))
        histograma_descritor = img_representacao.histograma_de_frequencia(descritor)
        Descritor.salvar_descritor(histograma_descritor, caminho, NOME_DESCRITOR)
                
print("extração de caracteriscas finalizada e descritores salvos!")

estração de caracteriscas finalizada e descritores salvos!


In [0]:
descritores_treinamento = np.empty((0,QUANTIDADE_PALAVRAS_VIRTUAIS))

for caminho in dados_treinamento:
    descritores_treinamento = np.append(descritores_treinamento, Descritor.carregar_descritores(caminho,NOME_DESCRITOR))

formato do array de descritores:  (400, 128)
formato do array de descritores:  (400, 128)


In [0]:
QUANTIDADE_DADOS_TREINAMENTO = 400
QUANTIDADE_DADOS_TESTE = 100
rotulos_treinamento = np.concatenate((np.ones(QUANTIDADE_DADOS_TREINAMENTO, dtype=np.uint8),np.zeros(QUANTIDADE_DADOS_TREINAMENTO, dtype=np.uint8)))
rotulos_teste = np.concatenate((np.ones(QUANTIDADE_DADOS_TESTE, dtype=np.uint8), np.zeros(QUANTIDADE_DADOS_TESTE, dtype=np.uint8)))

In [28]:
dados_teste = ['Aula/Teste/positivos/', 'Aula/Teste/negativos/']

descritores_teste = np.empty((0,QUANTIDADE_PALAVRAS_VIRTUAIS), dtype=np.uint8)

for caminho in dados_teste:
    
    for arquivo in arquivo_diretorio(caminho):
        img_descritor = Descritor.get_SIFT_descritores(os.path.join(caminho,arquivo))
        img_descritor = img_representacao.histograma_de_frequencia(img_descritor)
        img_dim_expandida = np.expand_dims(img_descritor, axis=0)
        descritores_teste = np.append(descritores_teste, img_dim_expandida, axis=0)
print("extração descritores teste finalizada")

extração descritores teste finalizada


In [31]:
print(rotulos_teste[150])
print(descritores_teste[150])

0
[ 2  4  5  2  7  7  7 11  2  4  1  3  6  5  1  1  1  1  0  2  4  7  4  2
  3  7  4  3  1  4  4  8  1  2  3  2  2  1  2  7  7  5  3  6  6  9  5  6
  2  5  8  0  2  4  1  8  5  2  4  7  1  1  3  1  3  7  4  4  0  3  4  2
  2  0  4  8  5 12  6  6  2  3  5  2  2  4  1 27  4  1  4  4  9  2  2  6
  7  4  0  5  1  2  5  2  1  9  3  7  2  4  1  2  3  6  7  4  7  6  2  5
  3  1  6  1  1  8  4  1]


In [0]:
from keras.models import Sequential
from keras.layers import Dense

In [0]:
mlp_model = Sequential([
    Dense(512, activation='relu', input_shape=(QUANTIDADE_PALAVRAS_VIRTUAIS,)),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

mlp_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])